# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 3:41PM,261255,22,646059,"NBTY Global, Inc.",Released
1,Apr 17 2023 3:40PM,261254,10,4.7.23Uniderm.1,"Uniderm USA, Inc",Released
2,Apr 17 2023 3:40PM,261252,10,PRF-58443,Bio-PRF,Released
3,Apr 17 2023 3:40PM,261252,10,PRF-58446,Bio-PRF,Released
4,Apr 17 2023 3:40PM,261252,10,PRF-58451,Bio-PRF,Released
5,Apr 17 2023 3:40PM,261252,10,PRF-58454,Bio-PRF,Released
6,Apr 17 2023 3:40PM,261252,10,PRF-58460,Bio-PRF,Released
7,Apr 17 2023 3:38PM,261253,10,131099,"Virtue Rx, LLC",Released
8,Apr 17 2023 3:35PM,261251,10,Uniderm.4.12.23.2,"Uniderm USA, Inc",Released
9,Apr 17 2023 3:34PM,261250,10,SONSB0002173,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,261251,Released,1
39,261252,Released,5
40,261253,Released,1
41,261254,Released,1
42,261255,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261251,NaN,NaN,1.0
261252,NaN,NaN,5.0
261253,NaN,NaN,1.0
261254,NaN,NaN,1.0
261255,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,0.0,6.0,47.0
261161,0.0,0.0,1.0
261179,0.0,1.0,0.0
261183,4.0,36.0,6.0
261193,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,0,6,47
261161,0,0,1
261179,0,1,0
261183,4,36,6
261193,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,0,6,47
1,261161,0,0,1
2,261179,0,1,0
3,261183,4,36,6
4,261193,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,,6,47
1,261161,,,1
2,261179,,1,
3,261183,4,36,6
4,261193,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 3:41PM,261255,22,"NBTY Global, Inc."
1,Apr 17 2023 3:40PM,261254,10,"Uniderm USA, Inc"
2,Apr 17 2023 3:40PM,261252,10,Bio-PRF
7,Apr 17 2023 3:38PM,261253,10,"Virtue Rx, LLC"
8,Apr 17 2023 3:35PM,261251,10,"Uniderm USA, Inc"
9,Apr 17 2023 3:34PM,261250,10,"Nextsource Biotechnology, LLC"
11,Apr 17 2023 3:32PM,261249,10,"Methapharm, Inc."
12,Apr 17 2023 3:30PM,261248,19,"GUSA Granules USA, Inc."
13,Apr 17 2023 3:29PM,261246,10,"Citieffe, Inc."
15,Apr 17 2023 3:28PM,261244,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 17 2023 3:41PM,261255,22,"NBTY Global, Inc.",,,1
1,Apr 17 2023 3:40PM,261254,10,"Uniderm USA, Inc",,,1
2,Apr 17 2023 3:40PM,261252,10,Bio-PRF,,,5
3,Apr 17 2023 3:38PM,261253,10,"Virtue Rx, LLC",,,1
4,Apr 17 2023 3:35PM,261251,10,"Uniderm USA, Inc",,,1
5,Apr 17 2023 3:34PM,261250,10,"Nextsource Biotechnology, LLC",,,2
6,Apr 17 2023 3:32PM,261249,10,"Methapharm, Inc.",,,1
7,Apr 17 2023 3:30PM,261248,19,"GUSA Granules USA, Inc.",,,1
8,Apr 17 2023 3:29PM,261246,10,"Citieffe, Inc.",,2,
9,Apr 17 2023 3:28PM,261244,19,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:41PM,261255,22,"NBTY Global, Inc.",1,,
1,Apr 17 2023 3:40PM,261254,10,"Uniderm USA, Inc",1,,
2,Apr 17 2023 3:40PM,261252,10,Bio-PRF,5,,
3,Apr 17 2023 3:38PM,261253,10,"Virtue Rx, LLC",1,,
4,Apr 17 2023 3:35PM,261251,10,"Uniderm USA, Inc",1,,
5,Apr 17 2023 3:34PM,261250,10,"Nextsource Biotechnology, LLC",2,,
6,Apr 17 2023 3:32PM,261249,10,"Methapharm, Inc.",1,,
7,Apr 17 2023 3:30PM,261248,19,"GUSA Granules USA, Inc.",1,,
8,Apr 17 2023 3:29PM,261246,10,"Citieffe, Inc.",,2,
9,Apr 17 2023 3:28PM,261244,19,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:41PM,261255,22,"NBTY Global, Inc.",1,,
1,Apr 17 2023 3:40PM,261254,10,"Uniderm USA, Inc",1,,
2,Apr 17 2023 3:40PM,261252,10,Bio-PRF,5,,
3,Apr 17 2023 3:38PM,261253,10,"Virtue Rx, LLC",1,,
4,Apr 17 2023 3:35PM,261251,10,"Uniderm USA, Inc",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:41PM,261255,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Apr 17 2023 3:40PM,261254,10,"Uniderm USA, Inc",1.0,NaN,NaN
2,Apr 17 2023 3:40PM,261252,10,Bio-PRF,5.0,NaN,NaN
3,Apr 17 2023 3:38PM,261253,10,"Virtue Rx, LLC",1.0,NaN,NaN
4,Apr 17 2023 3:35PM,261251,10,"Uniderm USA, Inc",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:41PM,261255,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Apr 17 2023 3:40PM,261254,10,"Uniderm USA, Inc",1.0,0.0,0.0
2,Apr 17 2023 3:40PM,261252,10,Bio-PRF,5.0,0.0,0.0
3,Apr 17 2023 3:38PM,261253,10,"Virtue Rx, LLC",1.0,0.0,0.0
4,Apr 17 2023 3:35PM,261251,10,"Uniderm USA, Inc",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4440942,49.0,39.0,4.0
12,261209,1.0,0.0,0.0
15,261216,29.0,0.0,0.0
16,261194,0.0,1.0,0.0
18,261161,1.0,0.0,0.0
19,3657202,35.0,12.0,0.0
20,522098,48.0,26.0,0.0
22,261255,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4440942,49.0,39.0,4.0
1,12,261209,1.0,0.0,0.0
2,15,261216,29.0,0.0,0.0
3,16,261194,0.0,1.0,0.0
4,18,261161,1.0,0.0,0.0
5,19,3657202,35.0,12.0,0.0
6,20,522098,48.0,26.0,0.0
7,22,261255,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,39.0,4.0
1,12,1.0,0.0,0.0
2,15,29.0,0.0,0.0
3,16,0.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,35.0,12.0,0.0
6,20,48.0,26.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,12,Released,1.0
2,15,Released,29.0
3,16,Released,0.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,39.0,0.0,0.0,1.0,0.0,12.0,26.0,0.0
Released,49.0,1.0,29.0,0.0,1.0,35.0,48.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,39.0,0.0,0.0,1.0,0.0,12.0,26.0,0.0
2,Released,49.0,1.0,29.0,0.0,1.0,35.0,48.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,39.0,0.0,0.0,1.0,0.0,12.0,26.0,0.0
2,Released,49.0,1.0,29.0,0.0,1.0,35.0,48.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()